In [1]:
import $ivy.`org.tpolecat::doobie-quill:0.8.8`
import $ivy.`io.getquill::quill-jdbc:3.5.1`
import $ivy.`org.postgresql:postgresql:42.2.10`
import $ivy.`org.tpolecat::doobie-postgres:0.8.8`
import $plugin.$ivy.`org.spire-math:kind-projector_2.13.0-RC1:0.9.10`

import $ivy.$                                 

import $ivy.$                              

import $ivy.$                                  

import $ivy.$                                    

import $plugin.$                                                     

In [2]:
import cats._, cats.syntax.all._, cats.data._, cats.instances.all._
import fs2.Stream
import cats.effect.IO, cats.effect.Blocker
import doobie._, doobie.implicits._
import _root_.io.getquill.{ idiom => _, _ }
import doobie.quill.DoobieContext

import cats._, cats.syntax.all._, cats.data._, cats.instances.all._

import fs2.Stream

import cats.effect.IO, cats.effect.Blocker

import doobie._, doobie.implicits._

import _root_.io.getquill.{ idiom => _, _ }

import doobie.quill.DoobieContext

In [3]:
val dc = new DoobieContext.Postgres(Literal) // Literal naming scheme
import dc._

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


dc: DoobieContext.Postgres[Literal.type] = doobie.quill.DoobieContext$Postgres@17ac624c
import dc._

In [4]:
import doobie.util.ExecutionContexts
implicit val cs = IO.contextShift(ExecutionContexts.synchronous)
val xa = Transactor.fromDriverManager[IO](
  "org.postgresql.Driver",     // driver classname
  "jdbc:postgresql:world",     // connect URL (driver-specific)
  "postgres",                  // user
  "",                          // password
  Blocker.liftExecutionContext(ExecutionContexts.synchronous) // just for testing
)

import doobie.util.ExecutionContexts

cs: effect.ContextShift[IO] = cats.effect.internals.IOContextShift@1d6c23a2
xa: Transactor.Aux[IO, Unit] = doobie.util.transactor$Transactor$$anon$13@39ba677a

In [5]:
val y = xa.yolo // a stable reference is required
import y._

y: doobie.util.yolo.Yolo[IO] = doobie.util.yolo$Yolo@19578943
import y._

In [6]:
import java.util.concurrent.TimeUnit

def timed[A](code: => A, nano: Boolean = false): A = {
    val start = System.nanoTime
    val result: A = code
    val end = System.nanoTime
    val nanos = end -start
    val millis = TimeUnit.NANOSECONDS.toMillis(nanos)
    println(if (nano) s"$nanos nanos" else s"$millis millis")
    result
}

import java.util.concurrent.TimeUnit


defined function timed

# Domain model

In [7]:
case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

defined class Country
defined class City

# SQL queries

In [8]:
sql"select code, name, capital  from country".query[Country].to[List].transact(xa).unsafeRunSync

res7: List[Country] = List(
  Country("AFG", "Afghanistan", Some(1)),
  Country("NLD", "Netherlands", Some(5)),
  Country("ANT", "Netherlands Antilles", Some(33)),
  Country("ALB", "Albania", Some(34)),
  Country("DZA", "Algeria", Some(35)),
  Country("ASM", "American Samoa", Some(54)),
  Country("AND", "Andorra", Some(55)),
  Country("AGO", "Angola", Some(56)),
  Country("AIA", "Anguilla", Some(62)),
  Country("ATG", "Antigua and Barbuda", Some(63)),
  Country("ARE", "United Arab Emirates", Some(65)),
  Country("ARG", "Argentina", Some(69)),
  Country("ARM", "Armenia", Some(126)),
  Country("ABW", "Aruba", Some(129)),
  Country("AUS", "Australia", Some(135)),
  Country("AZE", "Azerbaijan", Some(144)),
  Country("BHS", "Bahamas", Some(148)),
  Country("BHR", "Bahrain", Some(149)),
  Country("BGD", "Bangladesh", Some(150)),
  Country("BRB", "Barbados", Some(174)),
  Country("BEL", "Belgium", Some(179)),
  Country("BLZ", "Belize", Some(185)),
  Country("BEN", "Benin", Some(187)),
  Count

In [9]:
sql"select code, name, capital from country".query[Country].check.unsafeRunSync

  Query0[cmd8.cmd6.Country] defined at cmd8.sc:1
  select code, name, capital from country
  ✓ SQL Compiles and TypeChecks
  ✓ C01 code    CHAR    (bpchar)  NOT NULL  →  String
  ✓ C02 name    VARCHAR (varchar) NOT NULL  →  String
  ✓ C03 capital INTEGER (int4)    NULL      →  Option[Int]


In [10]:
timed(sql"""|
    | select C.name, X.name 
    | from city as C, country as X 
    | where C.id = X.capital and C.population > 8000000""".stripMargin
    .query[(String, String)]
    .to[List]
    .transact(xa)
    .unsafeRunSync)

28 millis


res9: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

# Pure case classes

In [82]:
def queryCC1(countries: List[Country], cities: Map[Int,City]): List[(String, String)] =
    for {
        Country(_, name, Some(capital)) <- countries
        city <- cities.get(capital).toList
        if city.population > 8000000
    } yield (name, city.name)

defined function queryCC1

In [83]:
def queryCC2(countries: List[Country], cities: List[City]): List[(String, String)] = 
    for {
        country <- countries
        city <- cities if country.capital.exists(_ == city.id)
        if city.population > 8000000
    } yield (country.name, city.name)

defined function queryCC2

In [84]:
val realCountries = sql"select code, name, capital from country"
    .query[Country].to[List].transact(xa).unsafeRunSync

val realCities = sql"select id, name, countryCode, population from city"
    .query[City].to[List].transact(xa).unsafeRunSync

val realMapCities = Map.from(realCities.map(city => (city.id, city)))

realCountries: List[Country] = List(
  Country("AFG", "Afghanistan", Some(1)),
  Country("NLD", "Netherlands", Some(5)),
  Country("ANT", "Netherlands Antilles", Some(33)),
  Country("ALB", "Albania", Some(34)),
  Country("DZA", "Algeria", Some(35)),
  Country("ASM", "American Samoa", Some(54)),
  Country("AND", "Andorra", Some(55)),
  Country("AGO", "Angola", Some(56)),
  Country("AIA", "Anguilla", Some(62)),
  Country("ATG", "Antigua and Barbuda", Some(63)),
  Country("ARE", "United Arab Emirates", Some(65)),
  Country("ARG", "Argentina", Some(69)),
  Country("ARM", "Armenia", Some(126)),
  Country("ABW", "Aruba", Some(129)),
  Country("AUS", "Australia", Some(135)),
  Country("AZE", "Azerbaijan", Some(144)),
  Country("BHS", "Bahamas", Some(148)),
  Country("BHR", "Bahrain", Some(149)),
  Country("BGD", "Bangladesh", Some(150)),
  Country("BRB", "Barbados", Some(174)),
  Country("BEL", "Belgium", Some(179)),
  Country("BLZ", "Belize", Some(185)),
  Country("BEN", "Benin", Some(187))

In [90]:
timed(queryCC2(realCountries, realCities))
timed(queryCC1(realCountries, realMapCities), nano=true)

26 millis
370556 nanos


res89_0: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)
res89_1: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

# Domain algebras with IO

In [11]:
class CityAlg(xa: Transactor[IO]){
    def getCityName(id: Int): IO[Option[String]] = 
        sql"select name from city where id = $id"
            .query[String].option.transact(xa)
    
    def getCityPopulation(id: Int): IO[Option[Int]] = 
        sql"select population from city where id = $id"
            .query[Int].option.transact(xa)
}

object CityAlg extends CityAlg(xa)

class CountryAlg(xa: Transactor[IO]){
    def getAllCountries: IO[List[Country]] = 
        sql"select code, name, capital from country"
            .query[Country].to[List].transact(xa)
    
    def getCountryName(id: Int): IO[Option[String]] =
        sql"select name from country where id = $id"
            .query[String].option.transact(xa)
    
    def getCountryCapital(id: Int): IO[Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Int].option.transact(xa)
}

object CountryAlg extends CountryAlg(xa)

defined class CityAlg
defined object CityAlg
defined class CountryAlg
defined object CountryAlg

In [12]:
val queryIO: IO[List[Option[(String, String)]]] = for {
    countries <- CountryAlg.getAllCountries
    result <- countries.traverse{

        case Country(code,name,Some(capital)) => 
            CityAlg.getCityName(capital).map(_.map((_, name)))
        case _ => None.pure[IO]
    }
} yield result

queryIO: IO[List[Option[(String, String)]]] = Bind(
  Async(cats.effect.internals.IOBracket$$$Lambda$3258/948182915@1315a613, false),
  ammonite.$sess.cmd11$Helper$$Lambda$3499/1016790129@592aa540
)

In [13]:
timed(queryIO.unsafeRunSync)

3439 millis


res12: List[Option[(String, String)]] = List(
  Some(("Kabul", "Afghanistan")),
  Some(("Amsterdam", "Netherlands")),
  Some(("Willemstad", "Netherlands Antilles")),
  Some(("Tirana", "Albania")),
  Some(("Alger", "Algeria")),
  Some(("Fagatogo", "American Samoa")),
  Some(("Andorra la Vella", "Andorra")),
  Some(("Luanda", "Angola")),
  Some(("The Valley", "Anguilla")),
  Some(("Saint John\u00b4s", "Antigua and Barbuda")),
  Some(("Abu Dhabi", "United Arab Emirates")),
  Some(("Buenos Aires", "Argentina")),
  Some(("Yerevan", "Armenia")),
  Some(("Oranjestad", "Aruba")),
  Some(("Canberra", "Australia")),
  Some(("Baku", "Azerbaijan")),
  Some(("Nassau", "Bahamas")),
  Some(("al-Manama", "Bahrain")),
  Some(("Dhaka", "Bangladesh")),
  Some(("Bridgetown", "Barbados")),
  Some(("Bruxelles [Brussel]", "Belgium")),
  Some(("Belmopan", "Belize")),
  Some(("Porto-Novo", "Benin")),
  Some(("Hamilton", "Bermuda")),
  Some(("Thimphu", "Bhutan")),
  Some(("La Paz", "Bolivia")),
  Some(("Sarajev

# Domain queries with Quill

In [13]:
/*def getCity(c: Option[Int]) = quote{
    query[City].filter(city => lift(c).exists(_ == city.id))
}*/

In [13]:
/*val q2 = quote { for {
    Country(_, name, Some(capital)) <- query[Country] 
//    city <- getCity(country.capital)
    city <- query[City] 
    if capital == city.id 
    if city.population > 8000000
} yield (name, city.name) }*/

In [14]:
val q2 = quote { for {
    country <- query[Country] 
//    city <- getCity(country.capital)
    city <- query[City] if country.capital.exists(_ == city.id) 
    if city.population > 8000000
} yield (country.name, city.name) }

q2: AnyRef with Quoted[Query[(String, String)]]{def quoted: io.getquill.ast.FlatMap;def ast: io.getquill.ast.FlatMap;def id47423137(): Unit;val liftings: Object} = querySchema("Country").flatMap(country => querySchema("City").filter(city => country.capital.exists((x1) => x1 == city.id)).filter(city => city.population > 8000000).map(city => (country.name, city.name)))

In [21]:
timed(run(q2).transact(xa).unsafeRunSync)

cmd20.sc:1: SELECT country.name, city.name FROM Country country, City city WHERE country.capital = city.id AND city.population > 8000000
val res20 = timed(run(q2).transact(xa).unsafeRunSync)
                     ^20 millis


res20: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

# Domain algebras with ConnectionIO

In [22]:
object CityAlgC{
    def getCityName(id: Int): ConnectionIO[Option[String]] = 
        sql"select name from city where id = $id"
            .query[String].option
    
    def getCityPopulation(id: Int): ConnectionIO[Option[Int]] = 
        sql"select population from city where id = $id"
            .query[Int].option
}

object CountryAlgC{
    def getAllCountries: ConnectionIO[List[Country]] = 
        sql"select code, name, capital from country"
            .query[Country].to[List]
    
    def getCountryName(id: Int): ConnectionIO[Option[String]] =
        sql"select name from country where id = $id"
            .query[String].option
    
    def getCountryCapital(id: Int): ConnectionIO[Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Int].option
}

defined object CityAlgC
defined object CountryAlgC

In [23]:
implicit class Ops[A](l: List[A]){
    def myTraverse[F[_]: Monad, B, C, D, E](
        cond: A => Option[B])(query: B => F[Option[C]])(
        filterSubject: B => F[Option[E]])(filterCond: E => Boolean)(
        result: A => C => D): F[List[D]] =
        l.traverse{ a => cond(a) match {
            case Some(b) => 
                filterSubject(b).flatMap{ 
                    case Some(t) if filterCond(t) => 
                        query(b).map(_.map(c => result(a)(c)))
                    case _ => 
                        Option.empty[D].pure[F]
                }
            case _ => Option.empty[D].pure[F]
        }}.map(_.collect{case Some(a) => a})
}

defined class Ops

In [24]:
val queryCIO: ConnectionIO[List[(String, String)]] = for {
    countries <- CountryAlgC.getAllCountries
    result <- countries.myTraverse(_.capital)(CityAlgC.getCityName)(
        CityAlgC.getCityPopulation)(_ > 8000000)(
        c => name => (name, c.name))
} yield result

queryCIO: ConnectionIO[List[(String, String)]] = FlatMapped(
  Suspend(
    BracketCase(
      Suspend(PrepareStatement("select code, name, capital from country")),
      doobie.hi.connection$$$Lambda$3250/1840349967@61c1d396,
      cats.effect.Bracket$$Lambda$3252/1944512446@b1a7509
    )
  ),
  ammonite.$sess.cmd23$Helper$$Lambda$3885/1137511907@998f11
)

In [34]:
timed(queryCIO.transact(xa).unsafeRunSync)

148 millis


res33: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

# Domain algebras with Streaming

In [36]:
object CityAlgS{
    def getCityName(id: Int): ConnectionIO[Option[String]] = 
        sql"select name from city where id = $id"
            .query[String].option
    
    def getCityPopulation(id: Int): ConnectionIO[Option[Int]] = 
        sql"select population from city where id = $id"
            .query[Int].option
}

object CountryAlgS{
    def getAllCountries: Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country"
            .query[Country].stream
    
    def getCountryName(id: Int): ConnectionIO[Option[String]] =
        sql"select name from country where id = $id"
            .query[String].option
    
    def getCountryCapital(id: Int): ConnectionIO[Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Int].option
}

defined object CityAlgS
defined object CountryAlgS

In [38]:
val querySCIO: Stream[ConnectionIO,(String, String)] = 
    CountryAlgS.getAllCountries.flatMap{ 
        case Country(code, name, Some(capital)) => 
            Stream.eval(CityAlgS.getCityPopulation(capital))
                .filter(_.exists(_ > 8000000))
                .flatMap{ _ => 
                    Stream.eval(CityAlgS.getCityName(capital)).flatMap{
                        case Some(cityName) => 
                            Stream.eval((cityName, name).pure[ConnectionIO])
                        case _ => 
                            Stream.empty
                    }}
        case _ => 
            Stream.empty
    }
    
/*result <- countries.myTraverse(_.capital)(CityAlgC.getCityName)(
        CityAlgC.getCityPopulation)(_ > 8000000)(
        c => name => (name, c.name))
} yield result*/

querySCIO: Stream[ConnectionIO, (String, String)] = Stream(..)

In [43]:
timed(querySCIO.compile.toList.transact(xa).unsafeRunSync)

180 millis


res42: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

In [44]:
implicit class WithFiT[F[_]: Applicative, A](st: Stream[F, A]){
    def withFilter(f: A => Boolean): Stream[F, A] = 
        st.flatMap{ a => 
            if (f(a)) Stream.eval(a.pure[F])
            else Stream.empty
        }
}

defined class WithFiT

In [45]:
val querySCIOF: Stream[ConnectionIO,(String, String)] = for {
    Country(_, name, Some(capital)) <- CountryAlgS.getAllCountries
    Some(population) <- Stream.eval(CityAlgS.getCityPopulation(capital))
    if population > 8000000
    Some(cityName) <- Stream.eval(CityAlgS.getCityName(capital))
} yield (cityName, name)

querySCIOF: Stream[ConnectionIO, (String, String)] = Stream(..)

In [50]:
timed(querySCIOF.compile.toList.transact(xa).unsafeRunSync)

160 millis


res49: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

In [51]:
object CityAlgS2{
    def getCityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def getCityPopulation(id: Int): Stream[ConnectionIO,Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream
}

object CountryAlgS2{
    def getAllCountries: Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country"
            .query[Country].stream
    
    def getCountryName(id: Int): Stream[ConnectionIO,String] =
        sql"select name from country where id = $id"
            .query[String].stream
    
    def getCountryCapital(id: Int): Stream[ConnectionIO, Int] =
        sql"select capital from country where id = $id"
            .query[Int].stream
}

defined object CityAlgS2
defined object CountryAlgS2

In [52]:
val querySCIO: Stream[ConnectionIO,(String, String)] = for {
    Country(_, name, Some(capital)) <- CountryAlgS2.getAllCountries
    population <- CityAlgS2.getCityPopulation(capital)
    if population > 8000000
    cityName <- CityAlgS2.getCityName(capital)
} yield (cityName, name)

querySCIO: Stream[ConnectionIO, (String, String)] = Stream(..)

In [62]:
timed(querySCIO.compile.toList.transact(xa).unsafeRunSync)

183 millis


res61: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

# MTL Algebras

In [63]:
trait CityAlg[F[_]]{
    def getCityName(id: Int): F[String]
    def getCityPopulation(id: Int): F[Int]
}

trait CountryAlg[F[_]]{
    def getAllCountries: F[Country]
    def getCountryName(code: String): F[String]
    def getCountryCapital(code: String): F[Int]
}

defined trait CityAlg
defined trait CountryAlg

In [64]:
implicit class WithFilterOpsCats[F[_]: FunctorFilter, A](f: F[A]){
    def withFilter(p: A => Boolean): F[A] = 
        f.filter(p)
}

defined class WithFilterOpsCats

In [66]:
def queryMTL[F[_]: Monad: FunctorFilter](implicit 
        Co: CountryAlg[F], Ci: CityAlg[F]): F[(String, String)] = for {
    Country(_, name, Some(capital)) <- Co.getAllCountries
    population <- Ci.getCityPopulation(capital)
    if population > 8000000
    cityName <- Ci.getCityName(capital)
} yield (cityName, name)

defined function queryMTL

In [67]:
implicit object CityAlgS3 extends CityAlg[λ[T => Stream[ConnectionIO, T]]]{
    def getCityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def getCityPopulation(id: Int): Stream[ConnectionIO,Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream
}

implicit object CountryAlgS3 extends CountryAlg[λ[T => Stream[ConnectionIO, T]]]{
    def getAllCountries: Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country"
            .query[Country].stream
    
    def getCountryName(code: String): Stream[ConnectionIO,String] =
        sql"select name from country where id = $code"
            .query[String].stream
    
    def getCountryCapital(code: String): Stream[ConnectionIO, Int] =
        sql"select capital from country where id = $code"
            .query[Int].stream
}

defined object CityAlgS3
defined object CountryAlgS3

In [68]:
implicit def MonadStream[F[_]: Applicative] = new Monad[Stream[F, ?]]{
    def flatMap[A, B](p: Stream[F, A])(f: A => Stream[F, B]) = 
        p.flatMap(f)
    def pure[A](x: A): fs2.Stream[F,A] =
        Stream.eval(x.pure[F])
    def tailRecM[A, B](a: A)(f: A => Stream[F,Either[A,B]]): Stream[F,B] =
        f(a).flatMap(_.fold(tailRecM(_)(f),Stream.emit)) // TBD
}

implicit def FunctorFilterStream[F[_]: Applicative] = new FunctorFilter[Stream[F, ?]]{
    def functor = new Functor[Stream[F, ?]]{
        def map[A, B](p: Stream[F, A])(f: A=>B) = 
            p.map(f)
    }
    def mapFilter[A, B](fa: Stream[F,A])(f: A => Option[B]): Stream[F,B] = 
        fa.collect{ a => f(a) match {
            case Some(b) => b
        }}

}

defined function MonadStream
defined function FunctorFilterStream

In [77]:
timed(queryMTL[λ[T => Stream[ConnectionIO, T]]].compile.toList.transact(xa).unsafeRunSync)

178 millis


res76: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

# Pure model with StateT

In [94]:
case class World(countries: Map[String, Country], cities: Map[Int, City])

defined class World

In [95]:
type WorldState[T] = State[World, List[T]]

implicit object CityWorldState extends CityAlg[WorldState]{
    def getCityName(id: Int): WorldState[String] = 
        StateT(world => Eval.now(world, 
             world.cities.get(id).map(city => city.name).toList))
    
    def getCityPopulation(id: Int): WorldState[Int] = 
        StateT(world => Eval.now(world, 
             world.cities.get(id).map(city => city.population).toList))
}

implicit object CountryWorldState extends CountryAlg[WorldState]{
    def getAllCountries: WorldState[Country] = 
        StateT(world => Eval.now(world, 
            world.countries.values.toList))

    def getCountryName(code: String): WorldState[String] =
        StateT(world => Eval.now(world, 
             world.countries.get(code).map(country => country.name).toList))
    
    def getCountryCapital(code: String): WorldState[Int] =
        StateT(world => Eval.now(world, 
             world.countries.get(code).flatMap(country => country.capital).toList))
}

defined type WorldState
defined object CityWorldState
defined object CountryWorldState

In [96]:
implicit def MonadStateList[F[_]: Monad: Traverse, S]: Monad[λ[T => State[S, F[T]]]] = 
    new Monad[λ[T => State[S, F[T]]]]{
        def flatMap[A, B](p: State[S, F[A]])(f: A => State[S, F[B]]) = 
            p.flatMap(_.traverse(f).map(_.flatten))

        def pure[A](x: A): State[S, F[A]] =
            x.pure[F].pure[State[S, ?]]
        
        def tailRecM[A, B](a: A)(f: A => State[S, F[Either[A,B]]]): State[S, F[B]] =
            ??? // TBD
    }

implicit def FunctorFilterStream[F[_]: Applicative: FunctorFilter, S] = 
    new FunctorFilter[λ[T => State[S, F[T]]]]{
        def functor = new Functor[λ[T => State[S, F[T]]]]{
            def map[A, B](p: State[S, F[A]])(f: A=>B) = 
                p.map(_.map(f))
        }
        def mapFilter[A, B](fa: State[S, F[A]])(f: A => Option[B]): State[S, F[B]] = 
            fa.map(_.mapFilter(f))
}

defined function MonadStateList
defined function FunctorFilterStream

In [97]:
val world: World =         
    World(Map("ES" -> Country("ES","Spain",Some(0)),
            "USA" -> Country("USA", "United States", Some(1)),
            "UK" -> Country("UK", "United Kingdom", Some(2)),
            "UNK" -> Country("UNK", "Unknown", None)),
        Map(0->City(0,"Madrid","ES",9000000),
            1->City(1,"Washington", "USA", 10000000),
            2->City(2,"London", "UK", 500000)))

world: World = World(
  Map(
    "ES" -> Country("ES", "Spain", Some(0)),
    "USA" -> Country("USA", "United States", Some(1)),
    "UK" -> Country("UK", "United Kingdom", Some(2)),
    "UNK" -> Country("UNK", "Unknown", None)
  ),
  Map(
    0 -> City(0, "Madrid", "ES", 9000000),
    1 -> City(1, "Washington", "USA", 10000000),
    2 -> City(2, "London", "UK", 500000)
  )
)

In [98]:
queryMTL[WorldState].run(world).value

res97: (World, List[(String, String)]) = (
  World(
    Map(
      "ES" -> Country("ES", "Spain", Some(0)),
      "USA" -> Country("USA", "United States", Some(1)),
      "UK" -> Country("UK", "United Kingdom", Some(2)),
      "UNK" -> Country("UNK", "Unknown", None)
    ),
    Map(
      0 -> City(0, "Madrid", "ES", 9000000),
      1 -> City(1, "Washington", "USA", 10000000),
      2 -> City(2, "London", "UK", 500000)
    )
  ),
  List(("Madrid", "Spain"), ("Washington", "United States"))
)

In [151]:
val realWorld = World(Map.from(realCountries.map(c => (c.code, c))), realMapCities)
timed(queryMTL[WorldState].runA(realWorld).value)

1 millis


realWorld: World = World(
  HashMap(
    "CYM" -> Country("CYM", "Cayman Islands", Some(553)),
    "MNG" -> Country("MNG", "Mongolia", Some(2696)),
    "SYR" -> Country("SYR", "Syria", Some(3250)),
    "LTU" -> Country("LTU", "Lithuania", Some(2447)),
    "GMB" -> Country("GMB", "Gambia", Some(904)),
    "UZB" -> Country("UZB", "Uzbekistan", Some(3503)),
    "MAC" -> Country("MAC", "Macao", Some(2454)),
    "KHM" -> Country("KHM", "Cambodia", Some(1800)),
    "ROM" -> Country("ROM", "Romania", Some(3018)),
    "BMU" -> Country("BMU", "Bermuda", Some(191)),
    "GUF" -> Country("GUF", "French Guiana", Some(3014)),
    "SWE" -> Country("SWE", "Sweden", Some(3048)),
    "ARM" -> Country("ARM", "Armenia", Some(126)),
    "LAO" -> Country("LAO", "Laos", Some(2432)),
    "SPM" -> Country("SPM", "Saint Pierre and Miquelon", Some(3067)),
    "GNQ" -> Country("GNQ", "Equatorial Guinea", Some(2972)),
    "USA" -> Country("USA", "United States", Some(3813)),
    "CXR" -> Country("CXR", "Christmas

# Tagless final approach

In [125]:
trait QUEΛ[Repr[_]]{
    // base types
    
    def bool(b: Boolean): Repr[Boolean]
    def int(i: Int): Repr[Int]
    def str(s: String): Repr[String]
    
    def >(i1: Repr[Int], i2: Repr[Int]): Repr[Boolean]
        
    // ADTs
    
    def tuple2[A, B](a: Repr[A], b: Repr[B]): Repr[(A, B)]
    
    def none[A]: Repr[Option[A]]
    def some[A](a: Repr[A]): Repr[Option[A]]
    def exists[A](o: Repr[Option[A]])(
        cond: Repr[A] => Repr[Boolean]): Repr[Boolean]
    
    // Comprehensions
    
    def bind[A, B](q: Repr[List[A]])(
        f: Repr[A] => Repr[List[B]]): Repr[List[B]]
    def pure[A](a: Repr[A]): Repr[List[A]]
    def where[A](cond: Repr[Boolean])(q: Repr[List[A]]): Repr[List[A]]
    
    // Equality
    
    def ===[A](a1: Repr[A], a2: Repr[A]): Repr[Boolean]
}

object QUEΛ{
    object syntax{   
        implicit class ComprehensionOps[Repr[_], A](la: Repr[List[A]])(implicit Q: QUEΛ[Repr]){
            def flatMap[B](cont: Repr[A] => Repr[List[B]]): Repr[List[B]] = 
                Q.bind(la)(cont)
            def map[B](f: Repr[A] => Repr[B]): Repr[List[B]] = 
                Q.bind(la)(a => Q.pure(f(a)))
            def withFilter(p: Repr[A] => Repr[Boolean]): Repr[List[A]] = 
                Q.bind(la)(a => Q.where(p(a))(Q.pure(a)))
        }

        implicit class BinOps[Repr[_], A](a1: Repr[A])(implicit Q: QUEΛ[Repr]){
            def ===(a2: Repr[A]): Repr[Boolean] = Q.===(a1, a2)
            def >(a2: Repr[Int])(implicit eq: Repr[A] =:= Repr[Int]): Repr[Boolean] = 
                Q.>(eq(a1), a2)
        }

        implicit def Tuple2QUEΛ[Repr[_], A, B](t2: (Repr[A], Repr[B]))(implicit Q: QUEΛ[Repr]): Repr[(A, B)] = 
            Q.tuple2(t2._1, t2._2)
        
        implicit class OptionOps[Repr[_], A](o: Repr[Option[A]])(implicit Q: QUEΛ[Repr]){
            def exists(cond: Repr[A] => Repr[Boolean]): Repr[Boolean] = 
                Q.exists(o)(cond)
        }

        implicit def IntQUEΛ[Repr[_]](i: Int)(implicit Q: QUEΛ[Repr]): Repr[Int] = 
            Q.int(i)
    }
    
    implicit def Std[S, F[_]: Monad]: QUEΛ[ReaderT[F, S, ?]] = new QUEΛ[ReaderT[F, S, ?]]{
        // BASE TYPES
        
        def bool(b: Boolean): ReaderT[F, S, Boolean] = 
            Kleisli.pure(b)
        def int(i: Int): ReaderT[F, S, Int] = 
            Kleisli.pure(i)
        def str(s: String): ReaderT[F, S, String] = 
            Kleisli.pure(s)
        
        def >(i1: ReaderT[F, S, Int], i2: ReaderT[F, S, Int]): ReaderT[F, S, Boolean] = 
            (i1, i2).mapN(_ > _)

        // ADTs
        
        def tuple2[A, B](a: ReaderT[F, S, A], b: ReaderT[F, S, B]): ReaderT[F, S, (A, B)] = 
            Applicative[ReaderT[F, S, ?]].tuple2(a,b)
        
        def none[A]: ReaderT[F, S, Option[A]] = 
            Kleisli.pure(None)
        def some[A](a: ReaderT[F, S, A]): ReaderT[F, S, Option[A]] = 
            a.map(Some(_))
        def exists[A](a: ReaderT[F, S, Option[A]])(
            cond: ReaderT[F, S, A] => ReaderT[F, S, Boolean]): ReaderT[F, S, Boolean] = 
            a.flatMap{ _.fold(Kleisli{ s: S => false.pure[F]}){
                a => cond(Kleisli.pure(a))
            }}
        
        // COMPREHENSIONS
        
        def bind[A, B](q: ReaderT[F, S, List[A]])(
                f: ReaderT[F, S, A] => ReaderT[F, S, List[B]]): ReaderT[F, S, List[B]] = 
            q.flatMap{ _.traverse((a: A) => 
                f(Kleisli.pure(a))).map(_.flatten)
            }
        def pure[A](a: ReaderT[F, S, A]): ReaderT[F, S, List[A]] = 
            a.map(List(_))
        def where[A](cond: ReaderT[F, S, Boolean])(q: ReaderT[F, S, List[A]]): ReaderT[F, S, List[A]] = 
            cond.flatMap{ if (_) q else Kleisli.pure(List()) }
        
        // EQUALITY
        
        def ===[A](a1: ReaderT[F, S, A], a2: ReaderT[F, S, A]): ReaderT[F, S, Boolean] = 
            (a1, a2).mapN(_ == _)
    }
}

defined trait QUEΛ
defined object QUEΛ

In [126]:
trait CityModel[Repr[_]]{
    def id(city: Repr[City]): Repr[Int]
    def name(city: Repr[City]): Repr[String]
    def country(country: Repr[City]): Repr[String]
    def population(city: Repr[City]): Repr[Int]
}

trait CountryModel[Repr[_]]{
    def code(country: Repr[Country]): Repr[String]
    def name(country: Repr[Country]): Repr[String]
    def capital(country: Repr[Country]): Repr[Option[Int]]
}

trait WorldModel[Repr[_]]{
    val Ci: CityModel[Repr]
    val Co: CountryModel[Repr]
    def countries: Repr[List[Country]]
    def cities: Repr[List[City]]
}

object WorldModel{
    
    object syntax{
        def countries[Repr[_]](implicit W: WorldModel[Repr]) = 
            W.countries
        def cities[Repr[_]](implicit W: WorldModel[Repr]) = 
            W.cities
        def name[Repr[_]](c: Repr[Country])(implicit W: WorldModel[Repr]) = 
            W.Co.name(c)
        def capital[Repr[_]](c: Repr[Country])(implicit W: WorldModel[Repr]) = 
            W.Co.capital(c)
        def id[Repr[_]](c: Repr[City])(implicit W: WorldModel[Repr]) = 
            W.Ci.id(c)        
        def cityName[Repr[_]](c: Repr[City])(implicit W: WorldModel[Repr]) = 
            W.Ci.name(c)        
        def population[Repr[_]](c: Repr[City])(implicit W: WorldModel[Repr]) = 
            W.Ci.population(c)
    }
    
    type WorldReader[T] = ReaderT[Eval, World, T]
    
    implicit object WorldModelStd extends WorldModel[WorldReader]{
        val Ci = new CityModel[WorldReader]{
            def id(city: WorldReader[City]): WorldReader[Int] = 
                city.map{_.id}
            def name(city: WorldReader[City]): WorldReader[String] = 
                city.map{_.name}
            def country(city: WorldReader[City]): WorldReader[String] = 
                city.map{_.countryCode}
            def population(city: WorldReader[City]): WorldReader[Int] = 
                city.map{_.population}
        }
        
        val Co = new CountryModel[WorldReader]{
            def code(country: WorldReader[Country]): WorldReader[String] = 
                country.map{_.code}
            def name(country: WorldReader[Country]): WorldReader[String] = 
                country.map{_.name}
            def capital(country: WorldReader[Country]): WorldReader[Option[Int]] = 
                country.map{_.capital}
        }
        
        def countries: WorldReader[List[Country]] = 
            Kleisli{ world => Eval.now(world.countries.values.toList) }
        
        def cities: WorldReader[List[City]] = 
            Kleisli{ world => Eval.now(world.cities.values.toList) }
    }
}

defined trait CityModel
defined trait CountryModel
defined trait WorldModel
defined object WorldModel

In [127]:
import QUEΛ.syntax._, WorldModel.syntax._

import QUEΛ.syntax._, WorldModel.syntax._

In [128]:
def queryTF[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- countries
        city <- cities if capital(country).exists(_ === id(city))
        if population(city) > 8000000
    } yield (cityName(city), name(country))

defined function queryTF

In [128]:
/*def query[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.bind(W.countries){ country => 
        Q.bind(W.cities){ city => 
            Q.where(Q.===(W.Co.capital(country), W.Ci.id(city))){
                Q.pure(Q.tuple2(W.Ci.name(city), W.Co.name(country)))
            }
        }
    }
  */  

In [0]:
timed(queryTF[WorldModel.WorldReader].apply(realWorld).value)

cmd0.sc:1: not found: value timed
val res0 = timed(queryTF[WorldModel.WorldReader].apply(realWorld).value)
           ^cmd0.sc:1: not found: value queryTF
val res0 = timed(queryTF[WorldModel.WorldReader].apply(realWorld).value)
                 ^cmd0.sc:1: not found: value WorldModel
val res0 = timed(queryTF[WorldModel.WorldReader].apply(realWorld).value)
                         ^cmd0.sc:1: not found: value realWorld
val res0 = timed(queryTF[WorldModel.WorldReader].apply(realWorld).value)
                                                       ^Compilation Failed

: 